<a href="https://colab.research.google.com/github/reenygeorge/FileManagementUtilities/blob/master/file_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone the GITHub repo { form-width: "20%" }

#@markdown **Note:** Please press on 'show code'; replace 'username' and 'access_token' in the code line with your saved settings and then press the _Play_ button to execute this cell.

#@markdown This mounts the github repo on which the relevant code reside. 

!git clone -b master https://github.com/reenygeorge/FileManagementUtilities


In [ ]:
#@title Manadatory set up code { form-width: "20%" }
%load_ext rpy2.ipython
%cd /content/

In [ ]:
#@title Manadatory set up code { form-width: "20%" }
%%R
source('/content/FileManagementUtilities/code/Revtools.R')


In [ ]:
#@title Convert RIS to CSV { form-width: "20%" }
#@markdown **Note:** Please name the input file as 'input_data.ris' and upload. The converted file will be named 'input_converted.csv' 

%%R
convert_ris_csv <- function(inFname, outFname){
  df <- read_bibliography(inFname)
  print(nrow(df))
  write.csv(df, outFname, row.names = FALSE)
}
convert_ris_csv('input_data.ris', 'input_converted.csv')

In [ ]:
#@title Convert CSV to RIS { form-width: "20%" }
#@markdown **Note:** Please name the input file as 'input_data.csv' and upload. The converted file will be named 'input_converted.ris' 
#@markdown **Note:** Column names should be appropriate (e.g. 'title', 'abstract' etc.)

%%R
convert_csv_ris <- function(inFname, outFname){
  df <- read.csv(inFname, stringsAsFactors = FALSE)
  colnames(df) <- tolower(colnames(df)) # Note: colnames should be in small letters!
  write_bibliography(df, outFname, format = "ris")
}

convert_csv_ris('input_data.csv', 'input_converted.ris')


In [ ]:
#@title concatenate multiple RIS to a single RIS { form-width: "20%" }
#@markdown **Note:** Please upload the ris files. All ris files will be concatenated, and the concatenated file will appear as 'merged'.ris.

%cat *.ris > merged.ris

In [ ]:
#@title Split a large csv to smaller files { form-width: "20%" }
#@markdown **Note:** Please edit the value of N (in the code below) to be number of rows needed in each file. If the input file is not an
#@markdown exact multiple of N, the last split file will have the remainder of the rows. 

#@markdown Both csv and ris versions of split files will be generated


%%R
# Provide value for N
N <- 10 # how many rows in each split portion. If an equal division is not possible, the last split will have remainder
df <- read.csv('input_data.csv', stringsAsFactors = FALSE)
nr <- nrow(df)
print(paste('Total rows in csv:', nr, sep=""))
df_split <- split(df, rep(1:ceiling(nr/N), each = N, length.out = nr))
for(i in 1:ceiling(nr/N)){
    file_csv <- paste('split', i, '.csv', sep="")
    write.csv(df_split[[i]], file_csv, row.names = FALSE)
    file_ris <- paste('split', i, '.ris', sep="")
    write_bibliography(df_split[[i]], file_ris, format = 'ris')
}


In [ ]:
#@title Remove the smaller set of data (input_subset.csv) from a larger csv (input_full.csv) { form-width: "20%" }
#@markdown Below code will give you input_full - input_subset (de-duping is done using eppi id)

%%R
full_db <- read_csv('input_full.csv')
print(nrow(full_db))
full_db[is.na(full_db)] <- ""
subset <- read_csv('input_subset.csv')
subset[is.na(subset)] <- ""
diff <- anti_join(full_db,subset, by="eppi_id")
write.csv(diff, "diff.csv", row.names = FALSE)
write_bibliographydiff, "diff.ris", format = "ris")